In [386]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [387]:
import sys
sys.path.insert(1, '../../')
import api

In [388]:
from better_profanity import profanity

In [389]:
from random import randint

In [390]:
cid = api.SPOTIFY_CID
secret = api.SPOTIFY_SKEY
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

# Start with eminem

In [391]:
eminem = 'https://open.spotify.com/artist/7dGJo4pcD2V6oG8kP0tJRR?si=cbkdp4ZLQl-iQ-FLN1-1Qg'

In [392]:
# get eminem's info
eminem_info = sp.artist(eminem)

In [393]:
eminem_info

{'external_urls': {'spotify': 'https://open.spotify.com/artist/7dGJo4pcD2V6oG8kP0tJRR'},
 'followers': {'href': None, 'total': 57317423},
 'genres': ['detroit hip hop', 'hip hop', 'rap'],
 'href': 'https://api.spotify.com/v1/artists/7dGJo4pcD2V6oG8kP0tJRR',
 'id': '7dGJo4pcD2V6oG8kP0tJRR',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eba00b11c129b27a88fc72f36b',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab67616100005174a00b11c129b27a88fc72f36b',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f178a00b11c129b27a88fc72f36b',
   'width': 160}],
 'name': 'Eminem',
 'popularity': 89,
 'type': 'artist',
 'uri': 'spotify:artist:7dGJo4pcD2V6oG8kP0tJRR'}

## get info of artists related to eminem

In [394]:
related_artists = sp.artist_related_artists(eminem)

In [395]:
related_artists.keys()

dict_keys(['artists'])

In [396]:
# returns 20 related artists
len(related_artists['artists'])

20

In [397]:
all_artists = [eminem_info] + related_artists['artists']

In [398]:
len(all_artists)

21

In [399]:
all_artists[0]['name']

'Eminem'

In [400]:
all_artists[0]['id']

'7dGJo4pcD2V6oG8kP0tJRR'

# Get top 10 tracks of each artist

In [401]:
def artist_top_tracks(artist_id):
    top_tracks_info = sp.artist_top_tracks(artist_id)
    song_names = [n['name'] for n in top_tracks_info['tracks']]
    return song_names

In [402]:
all_tracks = []
for artist in all_artists:
    artist_name = artist['name']
    artist_id = artist['id']
    songs = artist_top_tracks(artist_id)
    track_list = [[artist_name,song] for song in songs]
    all_tracks = all_tracks + track_list

# Get Lyric Data

In [410]:
from lyricsgenius import Genius
import re
import time

In [404]:
# initialize token
genius = Genius(api.GENIUS_TOKEN)

In [405]:
lyrics_list = []
finished_songs = []

In [412]:
for track in all_tracks:
    artist = track[0]
    song = track[1]
    
    # check if song is finished
    if [artist,song] in finished_songs:
        #print("{}{} finished".format(artist,song))
        continue
    try:
        genius_song = genius.search_song(song,artist)
    except:
        time.sleep(2.4) # adding some time helps retrieve 
        genius_song = genius.search_song(song,artist)
        
    if genius_song==None:
        finished_songs.append([artist,song])
        continue

    lyrics_list.append([artist,song,genius_song.lyrics])
    finished_songs.append([artist,song])

In [413]:
len(lyrics_list)

204

# Process Lyric Data

In [435]:
len(lyrics_list[0][2])

4310

In [436]:
def preprocess_lyrics(lyrics):
    lyrics = re.sub('^[^\[]*','',lyrics)  #get rid of everything before first verse
    lyrics = re.sub('\[(.*?)\]','[VERSE]',lyrics) # sub all [] with verse
    lyrics = re.sub('\((.*?)\)','',lyrics) # remove all parenthesis
    lyrics = re.sub('[0-9]+Embed','',lyrics) # remove the last Embed thingy
    lyrics = re.sub('  *\n','\n',lyrics) # remove space before newline
    lyrics = re.sub('[,"?.\']','',lyrics) #remove comma, quotes,questionmark,period
    lyrics = re.sub('\n+','\n',lyrics) # replace double newlines with just newline
    return lyrics

In [437]:
processed_lyrics = []
for track in lyrics_list:
    artist = track[0]
    song = track[1]
    lyric = track[2]
    processed_lyric = preprocess_lyrics(lyric)
    #processed_lyric = re.sub('\n\n','\n',processed_lyric) # replace double newlines with just newline
    processed_lyrics.append(processed_lyric)

## Build corpus & tokenize & create feed for model

In [438]:
#def data_preparation(lyric):

In [2]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences,to_categorical

In [439]:
corpus = []
for lyric_ in processed_lyrics:
    corpus += lyric_.lower().split("\n")
    
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

total_words = len(tokenizer.word_index) + 1

In [440]:
total_words, len(corpus)

(10058, 19188)

In [441]:
# break down each line into multiple training data
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)     

In [442]:
# pad sequences to match length
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,   
                      maxlen=max_sequence_len, padding='pre'))


In [443]:
# use the last word as label for model to predict
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = to_categorical(label, num_classes=total_words)


In [444]:
#    return predictors, label, max_sequence_len, total_words

## Create model & train

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dropout,Dense

In [445]:
def create_model(predictors, label, max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_len))
    model.add(LSTM(150))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.fit(predictors, label, epochs=100, verbose=1)
    return model

In [446]:
model = create_model(predictors, label, max_sequence_len, total_words)

Epoch 1/100
3664/3664 [==============================] - 155s 42ms/step - loss: 6.7366
Epoch 2/100
3664/3664 [==============================] - 154s 42ms/step - loss: 6.3308
Epoch 3/100
3664/3664 [==============================] - 156s 42ms/step - loss: 5.9821
Epoch 4/100
3664/3664 [==============================] - 156s 43ms/step - loss: 5.6170
Epoch 5/100
3664/3664 [==============================] - 158s 43ms/step - loss: 5.2858
Epoch 6/100
3664/3664 [==============================] - 162s 44ms/step - loss: 4.9801
Epoch 7/100
3664/3664 [==============================] - 159s 43ms/step - loss: 4.6988
Epoch 8/100
3664/3664 [==============================] - 156s 43ms/step - loss: 4.4493
Epoch 9/100
3664/3664 [==============================] - 157s 43ms/step - loss: 4.2288
Epoch 10/100
3664/3664 [==============================] - 157s 43ms/step - loss: 4.0366
Epoch 11/100
3664/3664 [==============================] - 164s 45ms/step - loss: 3.8640
Epoch 12/100
3664/3664 [=================

In [447]:
def generate_text(seed_text, next_words, max_sequence_len, model):
    for j in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen= 
                             max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
  
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == np.argmax(predicted[0]):
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [465]:
rap_lyrics = generate_text("Rap God", 100, max_sequence_len,model).split(" ")

In [466]:
title_len = 2
title = ""
for i in range(title_len):
    title += rap_lyrics.pop(0) + " "
title = title.strip()

In [467]:
verses = []
while len(rap_lyrics)>0:
    verse = ''
    for i in range(randint(8,12)):
        if len(rap_lyrics)==0:
            break
        verse += rap_lyrics.pop(0) + " "
    verse = verse.strip() + "\n"
    verse = profanity.censor(verse)  # censor out profanity
    verses.append(verse) 

In [469]:
print("TITLE: {}".format(title))
print(" ")
print("LYRICS:")
print(*verses, sep = "\n")

TITLE: Rap God
 
LYRICS:
was a one **** but you like me

when you broke for me givin it like

its hot work for the world i can do it gon be

doin me who pick it back back up back

back is the reality has goes to gettin trouble

it its not the life to trouble me in the nighttime

fame right like me than ever violence are or laugh in

the fact you that were not bout to be their

last its probably as a dog think its not supposed

to get a job **** over it get problems

out your first

